In [5]:
import numpy as np
from tqdm import tqdm
from sklearn.neighbors import KNeighborsClassifier

In [6]:
def read_npy_files_as_matrix(data_split):
    """
    Data must be in ./mimic_cxr/raw_embeddings
    """
    data_path = 'mimic_cxr/raw_embeddings/' + data_split + '/'
    if data_split == 'train':
        n_split = 17
    else:
        n_split = 1

    image_embeddings = []
    captions = []
    
    for file_index in tqdm(range(n_split)):
        feat_maps = np.load(data_path + 'feature_maps_' + str(file_index) + '.npy')[:500]
        caps = np.load(data_path + 'captions_' + str(file_index) + '.npy')[:500]
        image_embeddings.append(feat_maps)
        captions.append(caps)
        if file_index == 2:
            break
    
    return np.vstack(image_embeddings), np.vstack(captions)

train_image_embeddings, train_captions = read_npy_files_as_matrix('train')
val_image_embeddings, _ = read_npy_files_as_matrix('val')
test_image_embeddings, _ = read_npy_files_as_matrix('test')


100%|██████████| 1/1 [00:01<00:00,  1.10s/it]


In [7]:
indices = [*range(train_image_embeddings.shape[0])]
one_nn = KNeighborsClassifier(n_neighbors=1)
one_nn.fit(train_image_embeddings, indices)

KNeighborsClassifier(n_neighbors=1)

In [16]:
train_image_embeddings.shape

(1500, 65536)

(64, 65536)

In [30]:
def predict(embeddings, train_captions, one_nn, batch_size=64):
    captions = []
    total_len = embeddings.shape[0]
    data_loader = np.array_split(embeddings, range(batch_size, total_len, batch_size))

    for j, batch in enumerate(tqdm(data_loader)):
        dists, indices = one_nn.kneighbors(batch)
        captions.extend([train_captions[i] for i in indices])
        print(f"caption size: {len(captions)}")
    captions = np.array(captions).reshape(embeddings.shape[0], -1)

    return captions

predicted_val_captions = predict(val_image_embeddings, train_captions, one_nn)

 12%|█▎        | 1/8 [00:00<00:06,  1.14it/s]

caption size: 64


 25%|██▌       | 2/8 [00:01<00:04,  1.34it/s]

caption size: 128


 38%|███▊      | 3/8 [00:02<00:03,  1.43it/s]

...

caption size: 384


 88%|████████▊ | 7/8 [00:04<00:00,  1.48it/s]

caption size: 448


100%|██████████| 8/8 [00:05<00:00,  1.43it/s]

caption size: 500


In [26]:
train_captions.shape

(1500, 402)

In [29]:
val_image_embeddings.shape

(500, 65536)

In [31]:
predicted_val_captions.shape

(500, 402)

In [32]:
predicted_val_captions

array([[   4, 1140,   75, ...,    1,    1,    1],
       [   4, 1370, 1003, ...,    1,    1,    1],
       [   4,  774,   75, ...,    1,    1,    1],
       ...,
       [   4, 1896,  820, ...,    1,    1,    1],
       [   4,   96,    0, ...,    1,    1,    1],
       [   4, 1896,  820, ...,    1,    1,    1]], dtype=int64)

In [33]:
np.__config__.show()

blas_mkl_info:
  NOT AVAILABLE
blis_info:
  NOT AVAILABLE
openblas_info:
    library_dirs = ['D:\\a\\1\\s\\numpy\\build\\openblas_info']
    libraries = ['openblas_info']
    language = f77
    define_macros = [('HAVE_CBLAS', None)]
blas_opt_info:
    library_dirs = ['D:\\a\\1\\s\\numpy\\build\\openblas_info']
    libraries = ['openblas_info']
    language = f77
    define_macros = [('HAVE_CBLAS', None)]
lapack_mkl_info:
  NOT AVAILABLE
openblas_lapack_info:
    library_dirs = ['D:\\a\\1\\s\\numpy\\build\\openblas_lapack_info']
    libraries = ['openblas_lapack_info']
    language = f77
    define_macros = [('HAVE_CBLAS', None)]
lapack_opt_info:
    library_dirs = ['D:\\a\\1\\s\\numpy\\build\\openblas_lapack_info']
    libraries = ['openblas_lapack_info']
    language = f77
    define_macros = [('HAVE_CBLAS', None)]


In [34]:
import os
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['OPENBLAS_NUM_THREADS']

'1'

In [35]:
os.environ['MKL_NUM_THREADS']

KeyError: 'MKL_NUM_THREADS'

In [43]:
arr = np.zeros((8, 3))
start = 0
for i in range(3):
    new_arr = np.random.rand(2 if i == 2 else 3, 3)
    stop = start + new_arr.shape[0]
    arr[start:stop,:] = new_arr
    print(arr)
    start = stop

[[0.62408487 0.8459755  0.42735158]
 [0.87521632 0.86722164 0.11315083]
 [0.29878248 0.23107984 0.82366781]
 [0.         0.         0.        ]
 [0.         0.         0.        ]
 [0.         0.         0.        ]
 [0.         0.         0.        ]
 [0.         0.         0.        ]]
[[0.62408487 0.8459755  0.42735158]
 [0.87521632 0.86722164 0.11315083]
 [0.29878248 0.23107984 0.82366781]
 [0.11801006 0.79136219 0.18433709]
 [0.81919197 0.01698527 0.89859927]
 [0.3075785  0.21464877 0.00350434]
 [0.         0.         0.        ]
 [0.         0.         0.        ]]
[[0.62408487 0.8459755  0.42735158]
 [0.87521632 0.86722164 0.11315083]
 [0.29878248 0.23107984 0.82366781]
 [0.11801006 0.79136219 0.18433709]
 [0.81919197 0.01698527 0.89859927]
 [0.3075785  0.21464877 0.00350434]
 [0.38713637 0.22149694 0.19068075]
 [0.39688697 0.85467192 0.69495606]]


In [50]:
np.array(np.load('predicted_val_captions.npy'), dtype=np.int16)

array([[   4, 1140,   75, ...,    1,    1,    1],
       [   4, 1370, 1003, ...,    1,    1,    1],
       [   4,  774,   75, ...,    1,    1,    1],
       ...,
       [   4, 1896,  820, ...,    1,    1,    1],
       [   4,   96,    0, ...,    1,    1,    1],
       [   4, 1896,  820, ...,    1,    1,    1]], dtype=int16)

In [9]:
import numpy as np
type(np.load('mimic_cxr/raw_embeddings/val/feature_maps_0.npy')[0][0])

numpy.float32

In [15]:
arr = np.zeros((1), dtype=np.float32)
arr[0] = np.load('mimic_cxr/raw_embeddings/val/feature_maps_0.npy')[0][0]
type(arr[0])

numpy.float32